# Optimization Part 1: Univariate Scalar Functions

The general concept of "optimization" is one of the most important that you'll encounter in engineering.  Generally speaking, your goal in optimization is to maximize or minimize *something*, and the types of things you may want to maximize or minimize take many, many forms. But the basic concepts of optimization are relatively simple, and the strategy we use is always roughly the same. You will construct a model that predicts the value you are trying to optimize: let's call it a "objective function", though you may also hear it referred to as a "loss function" or a "cost function" in minimization problems. In general, we will let our "objective function" depend on multiple inputs, and our goal is to find the value(s) of those inputs that either minimize or maximize the value of our objective function.

What are some concrete examples of things you might want to maximize or minimize? 

How about profit? The profit obtained by a process is typically a complex function of raw material costs, sale prices, volume sold, capital costs, and operating costs. A very common goal is to figure out the scale your process should operate at (i.e., what volume) in order to maximize profit.  This is an example of an objective function that we'd like to maximize by finding the volume that gives us the best balance of revenue and expenses.  It is not always evident what that solution is since objective functions can be very complex. Moreover, we may be bound by several constraints on the system--it might be essential that we supply at least 100,000 units to meet our current orders. There are many strategies for optimizing the value of an objective function, but here we will focus on using numerical methods to optimize nonlinear problems. If you need to optimize a linear system, consider taking a course in linear algebra and linear programming methods.

Another example might be a new medical device - perhaps its overall cost to the patient is a complex function of size, materials and power supply...but you also have the constraint that you are providing the device for use in remote locations that don't have reliable access to medical care...so it needs to have a lifetime of at least 5 years because it cannot be replaced easily. You may need to minimize that cost by varying materials of construction, power supply, and size while still meeting the constraint that you can only replace it once every few years due to sparse availability of medical care in the area where you're working.

These are just examples to get you thinking about how frequently we are looking to either minimize or maximize a function. Again, that objective function may represent the cost of your product, the profit from your technology, the toxicity of a drug, or almost any other quantity you might want to either minimize or maximize. 

Because this is such a common goal, optimization is an essential skill for anyone working with data analysis and modelling, and there is a nice suite of optimization tools already built into Julia. To use them, all we really need to know is how to define an objective function and tweak our solver options. But first, we'll learn a little about how the methods work because it is always good to know what is under the hood. In this Assignment, we will focus on optimizing a value returned by a univariate *scalar* function, i.e., a model that has a single input and returns a single output.

## Locating extrema in a function

Let's start with a basic idea: we are trying to find extrema (either a minimum or a maximum value) returned by a univariate scalar function. We already learned how to do this analytically in our calculus courses, right?  We just find where the derivative of our function is equal to zero!

It's always nice to start out with a problem that we can solve analytically, so let's look at a cubic polynomial. 

$$y(x) = -1.6x^3 + 5x^2 + 8x - 23$$

We know that we can expect, at most, 3 real roots for this polynomial, and we can expect at most 2 extrema (either maxima or minima). In the cell below, graph this function on the domain $x = [-2.3, 4]$; plot the x-axis to visualize the root locations, and find any roots in that domain using an algorithm of your choice from Assignment 08.

According to my results, the roots of this function are x = -2.183, x = 1.978, and x = 3.330. From the graph, I can see clearly that these roots are entirely different from the locations of the extrema -- the maxima and minima. This is just to drive home the point that root finding (Assignment 08) and optimization (this Assignment) are two entirely different things. Looking at the graph, I can see that there is a minimum somewhere around $x = -1$, where $y \approx -25$ or so.  There is also a maximum somewhere around $x = 3$, where $y \approx 3$. 

**Important Note:** These locations are what we refer to as *local minima* and *local maxima*.  They are the lowest and highest values in their immediate vicinity, but it is clear that if we consider the function at x < -2.23 and x > 4, we can find larger and smaller values of y. The absolute minimum and maximum value of a function would be referred to as *global optima*.  For this function, there are no global optima as the function increases continuously below x = -1 or so, and it decreases continuous above x = 3 or so.

While a graphical analysis is usually informative, we can be more precise in our solution by finding the roots of the derivative of y(x), i.e., by solving:

$$y^{\prime}(x) = -4.8x^2 + 10x + 8 = 0$$

Take your pick of solution method and solve this equation in the cell below.

I find that the two roots of the $y^\prime(x)$ are $x = -0.617$ and $x = 2.7$. At these locations $y^{\prime}(x) = 0$. Further, I can evaluate the function here: $y(-0.617) = -25.66$ and $y(2.7) = 3.56$. To determine if it is a maximum or a minimum in calculus, we would consider the second derivative.  At $x = -0.617$, $y^{\prime\prime}(x) = 15.9$, which indicates that the slope is increasing so this is a local minimum. At $x = 2.7$, $y^{\prime\prime}(x) = -15.9$, which means the slope is decreasing at this point and we have a local maximum. This is consistent with our graphical analysis. 

Great! We've fully explored the analytical solution! Let's make sure our numerical methods match up. We'll start by writing our own optimization algorithm, which gives us some insight into how this works.  

Newton was a super rad dude. So it's not surprising that he also has a method for finding the minimum of a function using an interative strategy. It is actually very similar to a Newton-Raphson iteration, except we use the first derivative and second derivative of the function instead of the first derivative and the function value. The general formula:

$$x_{i+1} = x_i - \frac{f^\prime(x_i)}{f^{\prime\prime}(x_i)}$$

Based on that expression, you can see that iterations will converge once the ***derivative*** of the function is zero, which is the criteria we typically look for in finding extrema in a function. In contrast, a Newton-Raphson root finding iteration converges when the ***function itself*** is zero.

Similar to a Newton-Raphson algorithm, this Newton's method looks like something we could implement in a loop! We won't worry right now about finding a maximum or minimum specifically, just extrema in general. To do that, all we need to do is let the while loop proceed until our derivative value is close enough to zero that we are satisfied.  It may not be completely clear how to do this just yet, so we'll walk through this example together.  Just as a refresher, we'll use shorthand function definitions for the function itself (y1), its first derivative (dy1), and its second derivative (ddy1):

    y1(x)   =   -1.6*x^3 + 5*x^2 + 8*x - 23
    dy1(x)  =   -3*1.6*x^2 + 10*x + 8
    ddy1(x) =   -2*3*1.6*x + 10

Now for the loop:

    x = 100.0
    while abs(dy1(x)) > 1e-8
        println([x, y1(x), dy1(x), ddy1(x)])
        x = x - dy1(x)/ddy1(x)
    end
    
Go ahead and run this for a couple of initial guesses and watch the algorithm find the extrema in this function.

As with root finding in the last exercise, I don't recommend we write our own minimization routines outside of doing so to learn more about how they work. In general, it can be difficult to get optimization routines to converge rapidly and stably, and more sophisticated methods are available in Julia. It is hard to say for certain, but I suspect 99% of our needs will be covered by the Optim.jl package.

https://julianlsolvers.github.io/Optim.jl/stable/#

We need to install the Optim.jl package and declare that we are using it.  You can do it in the notebook or in Julia at the command line.

    using Pkg
    Pkg.add("Optim")

**Important Note:** Our very simple Newton method above found extrema without distinguishing between maxima and minima.  It was only looking for a place where the derivative = 0. Julia's optimization algorithms will always *minimize* your objective function.  If you want to *maximize* a function, you'll have to multiply your objective function by -1.  

The usage for Julia's optimization algorithms is very similar to what we learned with root finding for scalar functions.  Let's start with the most basic option:  minimizing a univariate, scalar function, i.e., the case we are currently considering:

$$y(x) = -1.6x^3 + 5x^2 + 8x - 23$$

Since y(x) is univariate, we are really just looking for the value of x where y is at a minimum. Typically, we'll use either a Brent's method or a Golden Section, both of which require you to provide bounds on the range you want to search. There is a function in the Optim.jl toolbox called **optimize**.  That's what we'll use here, with syntax below.

    optimize(function_name, lowerbound, upperbound, method)
    
For example, to use Brent's on the range x = [-3,1]:

    optimize(y1, -3, 1, Brent())
    
We'll add one more thing: optimize returns a solution structure, so I usually store the results in a new variable:

    solution = optimize(y1, -3, 1, Brent())

This allows you to recall specific aspects of the solution using the solutionname.field syntax, e.g.:

    xmin = solution.minimizer
    ymin = solution.minimum

As with fzero, you may sometimes with to change the default options for optimize; you can do so using keywork arguments.

For example, here we set xtolerance to 1e-6, function tolerance to 1e-e, method to GoldenSection, and the maximum number of iterations to 2000.

    solution = optimize(y1, -3, 1, GoldenSection(), x_tol = 1e-6, f_tol = 1e-8, iterations = 2000)